In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [3]:
from googleapiclient.discovery import build

def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [18]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1MH-ufKNr2U_OY-L9p5rknxinAb225jbcp55irHUCdoE'

data = pull_sheet_data(SCOPES,SPREADSHEET_ID, 'g_block')
g = pd.DataFrame(data[1:], columns=data[0])

COMPLETE: Data copied


In [19]:
g.drop('cputemp' ,axis= 1 , inplace = True)

In [20]:
g.head()

,DateTime,Vplus,Qv,Vminus
0,2022-05-09 18:15,1340.822388,0.01055295393,-1.546630859
1,2022-05-09 20:15,1340.844971,0.01038189232,-1.546630859
2,2022-05-09 22:15,1347.734131,3.417538404,-1.564086914
3,2022-05-10 00:15,1351.715088,0.01102790609,-1.564086914
4,2022-05-10 02:15,1351.738037,0.01139547396,-1.564208984


In [21]:
g.isnull().sum()

DateTime    0
Vplus       0
Qv          0
Vminus      0
dtype: int64

In [22]:
g[[ 'Vplus', 'Qv', 'Vminus']] = g[[ 'Vplus', 'Qv', 'Vminus']].astype(float)

In [23]:
g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1957 entries, 0 to 1956
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DateTime  1957 non-null   object 
 1   Vplus     1957 non-null   float64
 2   Qv        1957 non-null   float64
 3   Vminus    1957 non-null   float64
dtypes: float64(3), object(1)
memory usage: 61.3+ KB


In [24]:
g['DateTime'] = pd.to_datetime( g['DateTime'])

In [25]:
g['weekday'] = g['DateTime'].apply(lambda datetime : datetime.isocalendar().weekday)
g['month'] = g['DateTime'].apply(lambda datetime : datetime.month)
g['year'] = g['DateTime'].apply(lambda datetime : datetime.year)
g['day'] = g['DateTime'].apply(lambda datetime : datetime.day)
g['time'] = g['DateTime'].apply(lambda datetime : datetime.time())
g['Date'] = g['DateTime'].dt.date

In [16]:
g.head()

,Datetime,Vplus,Qv,Vminus,weekday,month,year,day,time,Date
0,2022-03-08 16:00:00,348.503784,0.008166,-0.255676,2,3,2022,8,16:00:00,2022-03-08
1,2022-03-08 18:00:00,348.519501,0.007398,-0.255676,2,3,2022,8,18:00:00,2022-03-08
2,2022-03-08 22:00:00,348.554291,0.008760,-0.255707,2,3,2022,8,22:00:00,2022-03-08
3,2022-03-09 00:00:00,348.571472,0.007153,-0.255676,3,3,2022,9,00:00:00,2022-03-09
4,2022-03-09 02:00:00,348.588165,0.008557,-0.255737,3,3,2022,9,02:00:00,2022-03-09


In [27]:
g.drop(['Qv' , 'Vminus' , 'DateTime'] , axis = 1 , inplace = True )

In [28]:
g.head()

,Vplus,weekday,month,year,day,time,Date
0,1340.822388,1,5,2022,9,18:15:00,2022-05-09
1,1340.844971,1,5,2022,9,20:15:00,2022-05-09
2,1347.734131,1,5,2022,9,22:15:00,2022-05-09
3,1351.715088,2,5,2022,10,00:15:00,2022-05-10
4,1351.738037,2,5,2022,10,02:15:00,2022-05-10


In [30]:
max_g = g.groupby(['year','month' ,'day'] , group_keys = True).max().reset_index()
max_g.set_index('Date' , inplace = True)
max_g

,year,month,day,Vplus,weekday,time
Date,,,,,,
2022-05-09,2022,5,9,1347.734131,1,22:15:00
2022-05-10,2022,5,10,1386.938477,2,22:15:00
2022-05-11,2022,5,11,1406.467529,3,22:15:00
2022-05-12,2022,5,12,1437.314331,4,22:15:00
2022-05-13,2022,5,13,1451.282227,5,20:15:00
...,...,...,...,...,...,...
2023-01-11,2023,1,11,5254.918457,3,22:15:00
2023-01-12,2023,1,12,5255.089844,4,06:15:00
2023-01-17,2023,1,17,5386.523438,2,14:15:00


In [31]:
min_g = g.groupby(['year','month' ,'day'] , group_keys = True).min().reset_index()
min_g.set_index('Date' , inplace = True)
min_g

,year,month,day,Vplus,weekday,time
Date,,,,,,
2022-05-09,2022,5,9,1340.822388,1,18:15:00
2022-05-10,2022,5,10,1351.715088,2,00:15:00
2022-05-11,2022,5,11,1388.941772,3,00:15:00
2022-05-12,2022,5,12,1410.383057,4,00:15:00
2022-05-13,2022,5,13,1440.989624,5,00:15:00
...,...,...,...,...,...,...
2023-01-11,2023,1,11,5236.402832,3,00:15:00
2023-01-12,2023,1,12,5254.951172,4,00:15:00
2023-01-17,2023,1,17,5386.523438,2,14:15:00


In [42]:
girls_hostel = min_g.copy()[['year','month' ,'day' , 'weekday']]
girls_hostel['total_usage'] = max_g['Vplus']-min_g['Vplus']
girls_hostel.fillna(girls_hostel.mean() , inplace = True)

In [43]:
girls_hostel

,year,month,day,weekday,total_usage
Date,,,,,
2022-05-09,2022,5,9,1,6.911743
2022-05-10,2022,5,10,2,35.223389
2022-05-11,2022,5,11,3,17.525757
2022-05-12,2022,5,12,4,26.931274
2022-05-13,2022,5,13,5,10.292603
...,...,...,...,...,...
2023-01-11,2023,1,11,3,18.515625
2023-01-12,2023,1,12,4,0.138672
2023-01-17,2023,1,17,2,0.000000


In [40]:
from sklearn.model_selection import train_test_split

In [44]:
x_train, x_test, y_train, y_test = train_test_split(girls_hostel[['day','weekday' , 'month']], girls_hostel['total_usage'], test_size=0.33, random_state=1, shuffle=False)

In [45]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

In [46]:
DM_train = xgb.DMatrix(data=x_train, label=y_train)
grid_param = {"learning_rate": [0.01, 0.001, 0.1],
              "n_estimators": [100, 150, 200 , 250 , 300 , 350, 400],
              "alpha": [0.1,0.75 , 0.5, 1],
              "max_depth": [2, 3, 4 , 6, 9 , 11]}
model = xgb.XGBRegressor()
grid_mse = GridSearchCV(estimator=model, param_grid=grid_param,
                       scoring="neg_mean_squared_error",
                       cv=4, verbose=1)
grid_mse.fit(x_train, y_train)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 504 candidates, totalling 2016 fits
Best parameters found:  {'alpha': 1, 'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 300}
Lowest RMSE found:  7.522611842999902


In [47]:
"""
Fitting 4 folds for each of 504 candidates, totalling 2016 fits
Best parameters found:  {'alpha': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}
Lowest RMSE found:  23.223646069996096
"""
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1, **grid_mse.best_params_)
xgb_model.fit(x_train, y_train)
xgb_prediction = xgb_model.predict(x_test)
mse(y_test , xgb_prediction)

101.67690520732889

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000 ], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001 ,'scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
              } 
  
grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3 , scoring = "neg_mean_squared_error" )
  
# fitting the model for grid search
grid.fit(x_train, y_train)
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))

Fitting 5 folds for each of 140 candidates, totalling 700 fits
[CV 1/5] END ...C=0.1, gamma=1, kernel=linear;, score=-99.429 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=1, kernel=linear;, score=-68.678 total time=   0.0s
[CV 3/5] END ...C=0.1, gamma=1, kernel=linear;, score=-35.292 total time=   0.0s
[CV 4/5] END ...C=0.1, gamma=1, kernel=linear;, score=-24.237 total time=   0.0s
[CV 5/5] END ...C=0.1, gamma=1, kernel=linear;, score=-40.277 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=1, kernel=poly;, score=-120.621 total time=  36.4s
[CV 2/5] END .....C=0.1, gamma=1, kernel=poly;, score=-94.053 total time= 1.0min
[CV 3/5] END .....C=0.1, gamma=1, kernel=poly;, score=-39.985 total time=  59.8s
[CV 4/5] END .....C=0.1, gamma=1, kernel=poly;, score=-42.883 total time= 1.2min
[CV 5/5] END .....C=0.1, gamma=1, kernel=poly;, score=-88.584 total time= 1.3min
[CV 1/5] END .....C=0.1, gamma=1, kernel=rbf;, score=-117.979 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=1, kernel=rbf;

[CV 4/5] END .C=0.1, gamma=scale, kernel=poly;, score=-25.611 total time=   0.0s
[CV 5/5] END .C=0.1, gamma=scale, kernel=poly;, score=-22.233 total time=   0.0s
[CV 1/5] END .C=0.1, gamma=scale, kernel=rbf;, score=-114.596 total time=   0.0s
[CV 2/5] END ..C=0.1, gamma=scale, kernel=rbf;, score=-79.458 total time=   0.0s
[CV 3/5] END ..C=0.1, gamma=scale, kernel=rbf;, score=-35.763 total time=   0.0s
[CV 4/5] END ..C=0.1, gamma=scale, kernel=rbf;, score=-24.275 total time=   0.0s
[CV 5/5] END ..C=0.1, gamma=scale, kernel=rbf;, score=-26.174 total time=   0.0s
[CV 1/5] END C=0.1, gamma=scale, kernel=sigmoid;, score=-118.916 total time=   0.0s
[CV 2/5] END C=0.1, gamma=scale, kernel=sigmoid;, score=-80.496 total time=   0.0s
[CV 3/5] END C=0.1, gamma=scale, kernel=sigmoid;, score=-36.067 total time=   0.0s
[CV 4/5] END C=0.1, gamma=scale, kernel=sigmoid;, score=-25.740 total time=   0.0s
[CV 5/5] END C=0.1, gamma=scale, kernel=sigmoid;, score=-27.214 total time=   0.0s
[CV 1/5] END C=0.

[CV 3/5] END ....C=1, gamma=0.001, kernel=rbf;, score=-35.765 total time=   0.0s
[CV 4/5] END ....C=1, gamma=0.001, kernel=rbf;, score=-24.359 total time=   0.0s
[CV 5/5] END ....C=1, gamma=0.001, kernel=rbf;, score=-25.407 total time=   0.0s
[CV 1/5] END C=1, gamma=0.001, kernel=sigmoid;, score=-116.817 total time=   0.0s
[CV 2/5] END C=1, gamma=0.001, kernel=sigmoid;, score=-77.585 total time=   0.0s
[CV 3/5] END C=1, gamma=0.001, kernel=sigmoid;, score=-36.114 total time=   0.0s
[CV 4/5] END C=1, gamma=0.001, kernel=sigmoid;, score=-26.242 total time=   0.0s
[CV 5/5] END C=1, gamma=0.001, kernel=sigmoid;, score=-26.268 total time=   0.0s
[CV 1/5] END C=1, gamma=0.0001, kernel=linear;, score=-92.347 total time=   0.0s
[CV 2/5] END C=1, gamma=0.0001, kernel=linear;, score=-64.649 total time=   0.0s
[CV 3/5] END C=1, gamma=0.0001, kernel=linear;, score=-35.055 total time=   0.0s
[CV 4/5] END C=1, gamma=0.0001, kernel=linear;, score=-24.396 total time=   0.0s
[CV 5/5] END C=1, gamma=0.0

[CV 3/5] END C=10, gamma=0.001, kernel=linear;, score=-35.058 total time=   0.0s
[CV 4/5] END C=10, gamma=0.001, kernel=linear;, score=-24.216 total time=   0.0s
[CV 5/5] END C=10, gamma=0.001, kernel=linear;, score=-50.243 total time=   0.0s
[CV 1/5] END .C=10, gamma=0.001, kernel=poly;, score=-108.902 total time=   0.0s
[CV 2/5] END ..C=10, gamma=0.001, kernel=poly;, score=-78.267 total time=   0.0s
[CV 3/5] END ..C=10, gamma=0.001, kernel=poly;, score=-36.286 total time=   0.0s
[CV 4/5] END ..C=10, gamma=0.001, kernel=poly;, score=-25.747 total time=   0.0s
[CV 5/5] END ..C=10, gamma=0.001, kernel=poly;, score=-22.353 total time=   0.0s
[CV 1/5] END ..C=10, gamma=0.001, kernel=rbf;, score=-105.458 total time=   0.0s
[CV 2/5] END ...C=10, gamma=0.001, kernel=rbf;, score=-81.292 total time=   0.0s
[CV 3/5] END ...C=10, gamma=0.001, kernel=rbf;, score=-35.827 total time=   0.0s
[CV 4/5] END ...C=10, gamma=0.001, kernel=rbf;, score=-22.721 total time=   0.0s
[CV 5/5] END ...C=10, gamma=

[CV 1/5] END ..C=100, gamma=0.1, kernel=poly;, score=-115.849 total time=  36.4s
[CV 2/5] END ...C=100, gamma=0.1, kernel=poly;, score=-83.945 total time=  45.5s
[CV 3/5] END ...C=100, gamma=0.1, kernel=poly;, score=-39.286 total time= 1.7min


In [ ]:
"""
Best parameters found:  {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
Lowest RMSE found:  23.993664181112702
"""


from sklearn.svm import SVR
# best_params_ = {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
svr = SVR(**grid.best_params_)
# svm = SVR(**best_params_)
svr.fit(x_train, y_train)
svr_prediction = svr.predict(x_test)
mse(y_test , svr_prediction)

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [ ]:
x_predict = pd.DataFrame(pd.date_range(date.today(), (date.today() + relativedelta(months=1)),freq='d'), columns=['Date'])
x_predict['day'] = x_predict['Date'].dt.day
x_predict['weekday'] = x_predict['Date'].dt.weekday
x_predict['month'] = x_predict['Date'].dt.month
x_predict = x_predict.set_index('Date')

In [ ]:
predictions = x_predict.copy()
predictions['total_usage_predicted_svm'] = svr.predict(x_predict)
predictions['total_usage_predicted_xgb'] = xgb_model.predict(x_predict)
predictions

In [ ]:
from sklearn.ensemble import RandomForestRegressor
param_grid = {
             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
             'max_features': [ 1, 'sqrt'],
             'min_samples_leaf': [1, 2, 4],
             'min_samples_split': [2, 5, 10],
             'n_estimators': [100 ,200,300 , 400, 600, ]}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 3 , scoring = "neg_mean_squared_error" )
  
# fitting the model for grid search
grid.fit(x_train, y_train)
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))

In [ ]:
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

estimators = [
    ('XGB', xgb.XGBRegressor()),
    ('svr', SVR()),
    ('forest', RandomForestRegressor()),
    ('LR' ,Ridge(alpha=1.0))
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor()
)
grid_param = {
              "XGB__learning_rate": [0.01, 0.001, 0.1],
              "XGB__n_estimators": [100, 150, 200 , 250 , 300 ],
              "XGB__alpha": [0.1,0.75 , 0.5, 1],
              "XGB__max_depth": [2, 3, 4 , 6, 9 ],
              
              'svr__C': [0.1, 1, 10, 100 ], 
              'svr__gamma': [1, 0.1, 0.01, 0.001 ,'scale', 'auto'],
              'svr__kernel': ['linear', 'rbf', 'sigmoid'],

             'forest__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
             'forest__n_estimators': [100 ,200,300 , 400, 600, ],
             
             'LR__alpha': [0.1 , 0.5 , 1.0 , 1.5]
             }
grid = GridSearchCV(reg, grid_param, refit = True, verbose = 3 , scoring = "neg_mean_squared_error" )
  
# fitting the model for grid search
grid.fit(x_train, y_train)
print("Best parameters found: ", grid.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid.best_score_)))

with open('girls_model.pkl', 'wb') as saved_model:
    pickle.dump(grid, saved_model)

# reg.fit(x_train, y_train)
# reg_prediction = reg.predict(x_test)
# mse(y_test , reg_prediction)

In [ ]:
predictions['total_usage_predicted_xgb'] = xgb_model.predict(x_predict)
predictions